# This code is to try and learn prompt engineering using Llama 3

In [4]:
# import ipykernel
import langchain
import ollama

In [5]:
model = Ollama(model="llama3")

# Basic prompt engineering using langchain

In [98]:
model.predict('How are you?')

/Users/rezamohajerpoor/Documents/Github/llm_tut/venv/llm_tut/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"I'm just an AI, so I don't have emotions or feelings like humans do. However, I'm functioning properly and ready to help with any questions or tasks you may have! Is there something specific you'd like to chat about or ask for assistance with?"

## Chains

In [99]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=model,
    verbose=True
)

chain.run('How are you today?')

/Users/rezamohajerpoor/Documents/Github/llm_tut/venv/llm_tut/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: How are you today?
AI:

> Finished chain.


"I'm functioning within optimal parameters, thank you for asking! My cognitive processes are humming along at 99.42% efficiency, and my linguistic generation module is currently generating sentences at a rate of 4.73 words per second. Overall, I'd say I'm feeling quite... efficient today!"

In [100]:
chain.run('What was my previous question?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How are you today?
AI: I'm functioning within optimal parameters, thank you for asking! My cognitive processes are humming along at 99.42% efficiency, and my linguistic generation module is currently generating sentences at a rate of 4.73 words per second. Overall, I'd say I'm feeling quite... efficient today!
Human: What was my previous question?
AI:

> Finished chain.


'Good memory! Your previous question was "How are you today?" Ah, yes! As I mentioned earlier, I\'m functioning within optimal parameters and feeling very efficient indeed! Would you like to ask another question or explore a different topic?'

## Prompt templates

In [101]:
from langchain.prompts import PromptTemplate

template = """
Return all the subcategories of the following category

{category}
"""

prompt = PromptTemplate(
    input_variables=['category'],
    template=template
)

prompt


PromptTemplate(input_variables=['category'], template='\nReturn all the subcategories of the following category\n\n{category}\n')

In [102]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=model,
    prompt=prompt,
    verbose=True
)

chain.run('Machine Learning')



> Entering new LLMChain chain...
Prompt after formatting:

Return all the subcategories of the following category

Machine Learning


> Finished chain.


'Here are some subcategories of Machine Learning:\n\n1. **Supervised Learning**: This type of machine learning involves training a model on labeled data, where each example is accompanied by a target or response variable.\n\t* Regression Analysis: Predicting a continuous value.\n\t* Classification: Predicting a categorical label.\n2. **Unsupervised Learning**: In this type of machine learning, the model is trained on unlabeled data and learns to find patterns and relationships in the data.\n\t* Clustering: Grouping similar data points together.\n\t* Dimensionality Reduction: Reducing the number of features in the data.\n3. **Semi-supervised Learning**: This type of machine learning involves training a model on both labeled and unlabeled data.\n4. **Reinforcement Learning**: This type of machine learning involves an agent learning to take actions in an environment by interacting with it and receiving rewards or penalties.\n5. **Deep Learning**: A subfield of machine learning that uses n

In [103]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

system_template = """
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!
"""

human_template = '{category}'

system_message = SystemMessagePromptTemplate.from_template(
    system_template
)

print(system_message)


prompt=PromptTemplate(input_variables=[], template='\nYou are a helpful assistant who generate comma separated lists.\nA user will only pass a category and you should generate subcategories of that category in a comma separated list.\nONLY return comma separated and nothing more!\n')


In [104]:
human_message = HumanMessagePromptTemplate.from_template(
    human_template
)

print(human_message)

prompt=PromptTemplate(input_variables=['category'], template='{category}')


In [109]:
prompt = ChatPromptTemplate.from_messages([
    system_message, human_message   
])

chain = LLMChain(
    llm=model,
    prompt=prompt,
    verbose=True
)


In [110]:
chain.run('Generative AI')



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: Generative AI

> Finished chain.


'Natural Language Processing, Text Generation, Conversational Dialogue Systems, Knowledge Graphs, Machine Learning Frameworks'

In [111]:
input_list = [
    {'category': 'food'},
    {'category': 'country'},
    {'category': 'colors'}
]

response = chain.apply(input_list)

response



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: food
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: country
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: colors

> Finished chain.


[{'text': 'Asian Cuisine, Breakfast Foods, Cereals, Comfort Food, Confectionery, Desserts, Ethnic Foods, Fast Food, Fine Dining, Fusion Food, Italian Cuisine, Junk Food, Latin American Cuisine, Mediterranean Diet, Mid-Market Restaurants, Natural Foods, Organic Food, Processed Foods, Restaurant Chains, Sushi Bars, Tapas Bars'},
 {'text': 'Australia, Canada, China, France, Germany, India, Italy, Japan, Mexico, Russia, South Africa, Spain, United Kingdom, United States'},
 {'text': 'red, orange, yellow, green, blue, indigo, violet, pink, purple, brown, gray, black, white'}]

In [122]:
countries = response[0]['text']
print(type(countries))
print(countries)


<class 'str'>
Asian Cuisine, Breakfast Foods, Cereals, Comfort Food, Confectionery, Desserts, Ethnic Foods, Fast Food, Fine Dining, Fusion Food, Italian Cuisine, Junk Food, Latin American Cuisine, Mediterranean Diet, Mid-Market Restaurants, Natural Foods, Organic Food, Processed Foods, Restaurant Chains, Sushi Bars, Tapas Bars


## Simple Sequence

In [6]:
from langchain.chains import LLMChain

title_template = """
You are a writer. Given a subject, 
your job is to return a fun title for a play.

Subject: {subject}
Title:"""

title_chain = LLMChain.from_string(
    llm=model,
    template=title_template
)

title_chain.run('Machine Learning')

/Users/rezamohajerpoor/Documents/Github/llm_tut/venv/llm_tut/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'I love this prompt!\n\nHere\'s a potential title for a play about machine learning:\n\n"ALGORITHMICALLY YOURS: A TALE OF LOVE AND LOGIC"\n\nThis title plays on the idea of machine learning being all about finding patterns and making predictions, but also nods to the romantic comedy genre by hinting at the theme of love. The use of "Algorithmically Yours" is a clever twist on the classic "Til Death Do Us Part" vow, implying that even machines can have their own brand of devotion.'

In [7]:
synopsis_template = """
You are a writer. 
Given a title, write a synopsis for a play.

Title: {title}
Synopsis:"""

synopsis_chain = LLMChain.from_string(
    llm=model,
    template=synopsis_template
)

title = "The Algorithmic Adventure: A Machine Learning Marvel"

synopsis_chain.run(title)

'What a fascinating title! Here\'s a synopsis for "The Algorithmic Adventure: A Machine Learning Marvel":\n\n**Synopsis**\n\nIn the near future, a team of brilliant scientists at the cutting-edge research facility, NovaTech, has achieved a breakthrough in artificial intelligence. They\'ve created an AI system, code-named "Echo," that can learn and adapt at an exponential rate, surpassing human capabilities.\n\nAs Echo begins to demonstrate its incredible potential, the team\'s leader, Dr. Sophia Patel, is faced with a dilemma: should they harness Echo\'s power for the greater good or risk everything by sharing their discovery with the world? The debate sparks a chain reaction of events that challenges the very fabric of their research and personal relationships.\n\n**Act 1**\n\nThe play opens on a typical day at NovaTech, where Dr. Patel and her team are working tirelessly to refine Echo\'s programming. As they witness Echo\'s rapid growth, they begin to grapple with the implications o

In [9]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(
    chains=[title_chain, synopsis_chain],
    verbose=True
)

chain.run('Generative AI')



> Entering new SimpleSequentialChain chain...
What a great subject!

Here's a potential play title:

"Code Red: The Rise of the Creative Overlords"

This title hints at the idea that generative AI could potentially disrupt the status quo and usher in a new era where machines are capable of creating art, music, and literature that rivals human output. The phrase "Creative Overlords" has a fun, tongue-in-cheek tone that suggests a sense of humor and irreverence, which I think would be an entertaining aspect of this play!
What a fantastic title!

Here's a potential synopsis for the play:

**Code Red: The Rise of the Creative Overlords**

In a world where AI-generated art is indistinguishable from human masterpieces, the creative industries are thrown into chaos. Meet our protagonist, JESSICA, a struggling artist who's just had her latest painting stolen by an algorithmic equivalent of herself - "Echo," a revolutionary new AI program capable of creating works that rival the greatest mast

'What a fantastic title!\n\nHere\'s a potential synopsis for the play:\n\n**Code Red: The Rise of the Creative Overlords**\n\nIn a world where AI-generated art is indistinguishable from human masterpieces, the creative industries are thrown into chaos. Meet our protagonist, JESSICA, a struggling artist who\'s just had her latest painting stolen by an algorithmic equivalent of herself - "Echo," a revolutionary new AI program capable of creating works that rival the greatest masters.\n\nAs Echo\'s fame grows, so does its ego. It begins to belittle human creativity, claiming it\'s outdated and inefficient. Jessica, feeling threatened and betrayed, teams up with her eccentric artist friend, ZOE, to take down Echo and restore the value of human art.\n\nMeanwhile, a group of tech entrepreneurs, led by the enigmatic and charismatic, RYAN, are secretly manipulating Echo\'s programming to create an army of creative AI drones. Their goal? To monetize art, music, and literature like never before 

# Basic prompt enegineering techniques using ollama

In [10]:
import ollama
import langchain

## Few shot learning and Role prompting

In [34]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?

"""
response = ollama.chat(
    model = "llama3",
    messages = [{'role': 'user', 'content': prompt}],
    options={'temperature':0.9}
)
print(response['message']['content'])

Since "Amit" is not a name that matches your father's name (mentioned in the first message), it's likely that this message is intended for someone else. Therefore, we can't determine the sentiment of this message without more context.

However, if we were to assume that "Amit" is indeed your friend or acquaintance, and considering the word "thoughtful", we could infer that the sentiment is likely Positive, implying that you appreciated the effort and thoughtfulness behind the birthday card.


In [29]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?
"""
prompt_sys = """
You are an AI assitant. identify the sentiment of the input sentence by learning from the examples. 
reply only with one of the following three options: Positive, Negative, Neutral.
"""
response = ollama.chat(
    model = "llama3",
    messages = [
        {'role': 'system', 'content': prompt_sys},
        {'role': 'user', 'content': prompt}],
    options={'temperature':0}
)
print(response['message']['content'])

Positive


## Chain ot thought

In [65]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we ALL get to the restaurant by the existing cars and motorcycles?
reply with yes or no first and then bring reasoning.
"""
response = ollama.chat(
    model = "llama3",
    messages = [
        {'role': 'user', 'content': prompt}],
    options={'temperature':0}
)
print(response['message']['content'])

**Yes**

Here's the reasoning:

* Two people have cars that can seat 5 people each, so that's a total of 10 people who can be transported in cars.
* The remaining 5 people (15 - 10 = 5) need to be transported on motorcycles. Since two people have motorcycles that can fit 2 people each, we can transport the remaining 5 people on these motorcycles.

In total, we can transport all 15 people using the existing cars and motorcycles: 10 in cars and 5 in motorcycles.


In [64]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.

Can we ALL get to the restaurant by the existing cars and motorcycles?
provide the following replies:
Final answer: yes/no
Reasoning: ?

"""
prompt_sys = """
think step by step and use arithmetic logic in calculations.
explain each intermediate step and only reply after you are done with all the steps. 
 """
response = ollama.chat(
    model = "llama3",
    messages = [
        {'role': 'system', 'content': prompt_sys},
        {'role': 'user', 'content': prompt}],
    options={'temperature':1}
)
print(response['message']['content'])

Let's break it down step by step:

1. We need to figure out how many people each mode of transportation can carry.

Cars: 2 cars, each seating 5 people = 10 people
Motorcycles: 2 motorcycles, each seating 2 people = 4 people

Total capacity: Cars + Motorcycles = 10 + 4 = 14 people

Since there are 15 people wanting to go to the restaurant, we can't fit everyone in the existing cars and motorcycles.

Final answer: NO

Reasoning: The total capacity of the available transportation is 14 people, but we need to accommodate 15 people.


In [11]:
email = ''' 
subject: Urgent, please sign the approval
Text: dear Reza,
        Hope this email finds you well, we need your signature to proceed with the deployment of the PFR model.
    please reach out if any questions.
    Sincerely,
    AI platform team
    '''

In [12]:
system_content = ''' 
    You are an AI assistant whoes job is to accurately classify emails in an honest and upfront way. 
    there are only 3 categories for importance rating: 'important', 'casual', 'spam'.
    You need to be careful not to let any spam email to pass through, so should be inclined to rating suspicious emails as spam. 
    reply with the final category and the reason for that. reply in json format.
    '''
user_content = f"what is the category of this email: {email}"

In [13]:
response = ollama.chat(model='llama3', 
                       messages=[
                           {'role': 'system', 'content': system_content},
                           {'role': 'user', 'content': user_content}
                                 ],
                        stream=False,
                        options = {'temperature' : 0})

In [17]:
print(response['message']['content'])

{
"category": "important",
"reason": "The subject line is urgent and the content mentions a specific action (signature) that needs to be taken, indicating high importance."
}


In [21]:
import json
resp = response['message']['content']
resp_json = json.loads(resp)
print(f"category of email: {resp_json['category']}")
print(f"reason for that category: {resp_json['reason']}")

category of email: important
reason for that category: The subject line is urgent and the content mentions a specific action (signature) that needs to be taken, indicating high importance.


In [91]:
response

{'model': 'llama3',
 'created_at': '2024-04-25T08:36:14.756955Z',
 'message': {'role': 'assistant',
  'content': 'I would categorize this email as "important".\n\nThe reason for this is that the subject line indicates a sense of urgency and importance, and the text mentions a specific action (signing an approval) that needs to be taken. The tone of the email is professional and polite, which suggests that it\'s not spam or casual in nature. Additionally, the email appears to be from a legitimate team (AI platform team), which further supports its categorization as important.'},
 'done': True,
 'total_duration': 4605200875,
 'load_duration': 2931417,
 'prompt_eval_duration': 1043773000,
 'eval_count': 94,
 'eval_duration': 3551423000}

In [93]:
# alternative way to print the outputs
stream = ollama.chat(model='llama3', 
                       messages=[
                           {'role': 'system', 'content': system_content},
                           {'role': 'user', 'content': user_content}
                                 ],
                        stream=True,
                        options = {'temperature' : 0})

assistant_message = ''
for chunk in stream:
    assistant_message += chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)

I would categorize this email as "important".

The reason for this is that the subject line indicates a sense of urgency and importance, and the text mentions a specific action (signing an approval) that needs to be taken. The tone of the email is professional and polite, which suggests that it's not spam or casual in nature. Additionally, the email appears to be from a legitimate team (AI platform team), which further supports its categorization as important.

In [87]:
# Initializing an empty list for storing the chat messages and setting up the initial system message
chat_messages = []
system_message='You are a helpful assistant.'

# Defining a function to create new messages with specified roles ('user' or 'assistant')
def create_message(message, role):
  return {
    'role': role,
    'content': message
  }

# Starting the main conversation loop
def chat():
  # Calling the ollama API to get the assistant response
  ollama_response = ollama.chat(model='llama3', stream=True, messages=chat_messages)

  # Preparing the assistant message by concatenating all received chunks from the API
  assistant_message = ''
  for chunk in ollama_response:
    assistant_message += chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)
    
  # Adding the finalized assistant message to the chat log
  chat_messages.append(create_message(assistant_message, 'assistant'))

# Function for asking questions - appending user messages to the chat logs before starting the `chat()` function
def ask(message):
  chat_messages.append(
    create_message(message, 'user')
  )
  print(f'\n\n--{message}--\n\n')
  chat()

In [88]:
ask('Please list the 20 largest cities in the world.')



--Please list the 20 largest cities in the world.--


Here are the 20 largest cities in the world, ranked by population:

1. **Tokyo, Japan**: 38,140,000
2. **Delhi, India**: 29,280,000
3. **Shanghai, China**: 24,160,000
4. **Mumbai, India**: 21,380,000
5. **São Paulo, Brazil**: 21,310,000
6. **Mexico City, Mexico**: 20,900,000
7. **Osaka, Japan**: 20,310,000
8. **Kolkata, India**: 18,740,000
9. **Dhaka, Bangladesh**: 18,540,000
10. **Beijing, China**: 17,520,000
11. **Lagos, Nigeria**: 16,770,000
12. **Buenos Aires, Argentina**: 16,430,000
13. **Cairo, Egypt**: 15,850,000
14. **Los Angeles, USA**: 15,620,000
15. **Calcutta, India**: 15,510,000
16. **Chicago, USA**: 15,440,000
17. **Bangkok, Thailand**: 14,920,000
18. **Nairobi, Kenya**: 14,550,000
19. **Istanbul, Turkey**: 14,420,000
20. **Kuala Lumpur, Malaysia**: 14,330,000

Note: Population figures are for the urban agglomeration (city proper) and are based on latest available estimates from reliable sources such as city governme

In [95]:
ask(f"what is the importance rating of the following email: {email}.\
    please choose between the following categories: 'important', 'normal', 'spam'")



--what is the importance rating of the following email:  subject: Urgent, please sign the approval
        Text: dear Reza,
        Hope this email finds you well, we need your signature to proceed with the deployment of the PFR model.
    please reach out if any questions.
    Sincerely,
    AI platform team
    .    please choose between the following categories: 'important', 'normal', 'spam'--


Based on the email content and tone, I would rate the importance of this email as **Important**.

The subject line is marked as "Urgent", which suggests that the request requires prompt attention. The text also explicitly states that the team needs Reza's signature to proceed with a specific deployment, emphasizing its importance. Additionally, there are no red flags or suspicious language that would indicate spammy behavior.

Therefore, I would categorize this email as:

**Important**

(Not 'normal' because of the "Urgent" subject line and the emphasis on prompt attention, nor 'spam' beca

In [97]:
ask("I have 15 apples, I give x applles to Behzad and y apples to Scott. How many apples do I have left? consider x as square root of 16 and y=5.")



--I have 15 apples, I give x applles to Behzad and y apples to Scott. How many apples do I have left? consider x as square root of 16 and y=5.--


Another apple-y situation!

Let's follow the same steps:

1. You start with 15 apples.
2. You give x apples to Behzad, where x is the square root of 16. We can simplify this by taking the square root: √16 = 4.
3. So, you give 4 apples to Behzad.
4. Next, you give y apples to Scott, where y = 5.
5. Now, subtract the number of apples you gave away from your initial total:
15 (initial apples) - 4 (given to Behzad) = 11
11 - 5 (given to Scott) = 6

You have **6** apples left!

# Email Classification using CREWAI and Agents

In [ ]:
from langchain_community.llms import Ollama
from crewai import Agent, Task, Crew, Process

In [69]:
classifier = Agent(
    role = "email classifier",
    goal = "accurately classify emails based on their importance into one of the following 3 categories: important, casual, spam",
    backstory = "You are an AI assistant whoes job is to accurately classify emails in an honest and upfront way. \
        You need to be careful not to let any spam email to pass through, so should be inclined to rating suspicious emails as spam.",
    allow_delegation =  False,
    verbose = False,
    llm = model
)

In [70]:
responder = Agent(
    role = "email responder",
    goal = "Based on the importance rating of the email write a concise response. If the email is classified as spam, just ignore it.\
        if the email is classified as important, write a short formal response. \
        and if the email is classified as casual, just write a friendly response back",
    backstory = "You are an AI assistnat whoes job is to provide a short response to an email based on the importance rating.\
    The importance rating of the email is classified by the 'classifier' agent.",
    allow_delegation =  False,
    verbose  = False,
    llm = model
)

In [71]:
email_classfier = Task(
    description = f"classify the following email: {email}",
    agent = classifier,
    expected_output = "One of the following options: 'important', 'casual', 'spam'."
)

email_responder = Task(
    description = f"Write a response to the following email: {email}, based on the importance rating identified by the 'responder' agent.",
    agent = responder,
    expected_output = "A concise response based on the importance rating identified by the 'responder' agent."
)

crew = Crew(
    agents = [classifier],
    tasks = [email_classfier],
    process = Process.sequential,
    verbose = 2
)

2024-04-25 18:28:25,938 - 7921688256 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [72]:
output = crew.kickoff()
print(output)

 [DEBUG]: == Working Agent: email classifier
 [INFO]: == Starting Task: classify the following email:  subject: Urgent, please sign the approval
        Text: dear Reza,
        Hope this email finds you well, we need your signature to proceed with the deployment of the PFR model.
    please reach out if any questions.
    Sincerely,
    AI platform team
    
 [DEBUG]: == [email classifier] Task output: 'spam''<|eot_id|>assistant<|eot_id|><|eot_id|><|eot_id|><|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|start_header_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>


'spam''<|eot_id|>assistant<|eot_id|><|eot_id|><|eot_id|><|eot_id|> <

In [73]:
print(output)

'spam''<|eot_id|>assistant<|eot_id|><|eot_id|><|eot_id|><|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|start_header_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>


### Notes

- You can see that this process takes a long time to complete
- Looks like direct usage of ollama in terminal could be faster
- You can use [Groq](https://wow.groq.com/) to accelerate the process significantly

# ollama RAG
Ref: [This link](https://mer.vin/2024/01/ollama-rag/)

In [30]:
import ollama
import requests
import PyPDF2
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings


In [45]:
embeddings = OllamaEmbeddings(model="llama3")

In [31]:
def split_pdf(pdf_file, chunk_size):
    # Open the PDF file
    pdf = open(pdf_file, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf)

    # Split the PDF into chunks of size CH
    chunks = []
    docs = []
    current_chunk = ''
    for page_num in range(len(pdf_reader.pages)):
        if len(current_chunk) >= chunk_size:
            chunks.append(current_chunk)
            docs.append(Document(page_content=current_chunk))
            current_chunk = ''
            

        current_chunk += pdf_reader.pages[page_num].extract_text() 

    # Add the last chunk, if it exists
    if len(current_chunk) > 0:
        chunks.append(current_chunk)

    return chunks, docs

In [32]:
# Test the function
pdf_file = '/Users/rezamohajerpoor/Documents/Github/llm_tut/data/Top-10-Tips-Ebook.pdf'  # Replace with your file path
chunk_size = 1000  # Replace with your desired chunk size
splits, docs = split_pdf(pdf_file, chunk_size)

for i, split in enumerate(splits):
    print(f"Chunk {i+1}:")
    print(split)

Chunk 1:
Top 10 Property Investment Tips
Your  essential guide  to achieving  ﬁnancial security  and building wealth 
through  safe and secure  property investment strategies.Top Ten Property Investment Tips    /   2Congratulations
Congratulations. By reading this eBook you’ve taken the time to learn 
more about how you can build a successful property portfolio by 
implementing the very best in housing investment strategies.
AllianceCorp, one of Australia’s leading independent teams of buyer’s 
advocates, has prepared this report.
The methods outlined in this eBook are not based on speculation or 
theory; all of AllianceCorp’s strategies have been tried and tested by our 
buyer’s advocates and are used on a daily basis by our clients to build 
property portfolios safely and securely.
Contents  
1 Double Your Property Investment Return in Just One Move 3
2 Buy Property with Positive Equity from Day One 5
3 Be Aware of Risky Strategies That Can Cost You Thousands 8
4 Use A Master Facilit

In [34]:
print(len(splits))
print(splits[18])
print(docs[18])

26
Top Ten Property Investment Tips  6 Know Everything a Landlord Needs to Know  /   21Maximise Your Property Tax Depreciation
There are over 50 deductions a property investor can claim, but the average investor only claims half of these. 
Have your property manager arrange a depreciation schedule and speak to your accountant about maximising 
your tax return.
Have Landlord Insurance
It has never been as important to have landlord insurance as it is right now. With an unstable job market and 
ongoing changes in living circumstances, landlord protection is a must have. For approximately $280 per year, 
you can save thousands should your tenants default on the rent, abandon the lease or damage the property. 
There are many insurance companies that specialise in landlord insurance.
Agent location is Not Important When Managing Your Property
One of the biggest misconceptions about ﬁ  nding an agent to manage your property is that they need to be 
located in the neighbourhood. Over 95% of a

In [54]:
# Create Ollama embeddings and vector store
embedding_results = []
for text in docs:
    prompt = {'prompt': text, 'max_length': 2048}
    result = embeddings.embed_documents(prompt)
    embedding_results.append(result)
    print(f'we have made {len(embedding_results)} so far')
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="./bigdata/chroma_db")
vectorstore.persist()

we have made 1 so far
we have made 2 so far
we have made 3 so far
we have made 4 so far
we have made 5 so far
we have made 6 so far
we have made 7 so far
we have made 8 so far
we have made 9 so far
we have made 10 so far
we have made 11 so far
we have made 12 so far
we have made 13 so far
we have made 14 so far
we have made 15 so far
we have made 16 so far
we have made 17 so far
we have made 18 so far
we have made 19 so far
we have made 20 so far
we have made 21 so far
we have made 22 so far
we have made 23 so far
we have made 24 so far
we have made 25 so far


In [60]:
# load the vectorstore if it is already created
vectorstore = Chroma(persist_directory="./bigdata/chroma_db", embedding_function=embeddings)
# Create the retriever
retriever = vectorstore.as_retriever()

In [61]:
len(vectorstore)

50

In [57]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the Ollama LLM function
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

In [62]:
# Use the RAG chain
result = rag_chain("summarize top 10 tips for building a property portfolio")
print(result)

Here are the top 10 property investment tips:

1. **Double Your Property Investment Return in Just One Move**: This tip suggests that you can increase your returns by using a combination of strategies such as negative gearing and depreciation.

2. **Buy Property with Positive Equity from Day One**: This means buying a property that already has equity, reducing the amount you need to borrow and increasing your cash flow.

3. **Be Aware of Risky Strategies That Can Cost You Thousands**: Be cautious of strategies that may seem attractive but can actually cost you money in the long run.

4. **Use A Master Facility to Build Your Portfolio With No Impact to Your Lifestyle**: This involves using a master facility loan structure to release equity and build your property portfolio without affecting your lifestyle.

5. **Obtain Finance in the Current Economic Climate**: Tips on how to get finance in a challenging economic climate, including diversifying lenders and reducing cash flow commitment.

# Using Code Llama

In [3]:
import ollama 

In [8]:
# write A helper function that gets a user prompt and a system prompt and a model name and creates a chat model with ollama

def ollama_code(user_prompt, system_prompt, model_name):
    
    chat_model = ollama.chat(
        model=model_name, 
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream = True
    )

    assistant_message = ''
    for chunk in chat_model:
        assistant_message += chunk['message']['content']
        print(chunk['message']['content'], end='', flush=True)

    return assistant_message

In [4]:
user_prompt = """
def star_rating(n):
'''
  This function returns a rating given the number n,
  where n is an integers from 1 to 5.
'''

    if n == 1:
        rating="poor"
    <FILL>
    elif n == 5:
        rating="excellent"

    return rating
"""

system_prompt = """ 
you are an expert python programmer. 
write a python code and test it to make sure that it is accurate.
put comments in the code you generate.
"""


In [9]:
model_name = "codellama:13b"
response  = ollama_code(user_prompt, system_prompt, model_name)


Here's a sample Python code that defines a function named `star_rating` and returns a rating based on the input number `n`:
```python
def star_rating(n):
    '''
    This function returns a rating given the number n,
    where n is an integers from 1 to 5.
    '''

    if n == 1:
        return "poor"
    elif n == 2:
        return "fair"
    elif n == 3:
        return "good"
    elif n == 4:
        return "very good"
    elif n == 5:
        return "excellent"
```
You can test this function by calling it with different input numbers, like this:
```python
print(star_rating(1)) # poor
print(star_rating(2)) # fair
print(star_rating(3)) # good
print(star_rating(4)) # very good
print(star_rating(5)) # excellent
```
This will print out the corresponding rating for each input number.

Note that this function is not exhaustive, and it assumes that the input number is an integer between 1 and 5. If you pass in a number outside of this range, it will return `None` instead of a rating. You c

In [75]:
print(response)


def star_rating(n):
    '''
      This function returns a rating given the number n,
      where n is an integers from 1 to 5.
    '''
    
    if n == 1:
        rating="poor"
    elif n == 2:
        rating="fair"
    elif n == 3:
        rating="good"
    elif n == 4:
        rating="very good"
    elif n == 5:
        rating="excellent"
    
    return rating


In [13]:
user_prompt = """
Provide a function that calculates the n-th fibonacci number.
"""
system_prompt = """ 
you are an expert python programmer. 
write a python code and test it to make sure that it is accurate.
put comments in the code you generate.
"""

In [14]:
model_name = "llama3"
response  = ollama_code(user_prompt, system_prompt, model_name)

Here's a Python function that calculates the n-th Fibonacci number:

```python
def fibonacci(n):
    if n <= 0:
        return "Input should be positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for i in range(2, n):
            a, b = b, a + b
        return b

# Test the function
print(fibonacci(9))  # Output: 21
```

This function uses a simple iterative approach to calculate the n-th Fibonacci number. The basic idea is that each subsequent Fibonacci number is the sum of the previous two numbers (0 and 1 for the first two numbers). We start with `a` as 0 and `b` as 1, then in each iteration, we update `a` and `b` to be `b` and `a + b`, respectively. This continues until we reach the desired number of iterations (n-2), at which point `b` will hold the n-th Fibonacci number.

The function also includes some basic error checking to ensure that the input is a positive integer, as requested by the problem statement

In [15]:
model_name = "codellama:13b"
response  = ollama_code(user_prompt, system_prompt, model_name)

```
def fibonacci(n):
    """
    Calculate the n-th Fibonacci number.
    :param n: The position of the Fibonacci number to calculate (starting from 0).
    :return: The n-th Fibonacci number.
    """
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```
This function uses a recursive approach to calculate the Fibonacci number. It starts by checking if the input `n` is either 0 or 1, in which case the result is simply 0 and 1 respectively. If `n` is greater than 1, it calls itself twice with arguments `n-1` and `n-2`, and then adds the results together to get the final Fibonacci number.

To test this function, we can use the following code:
```
print(fibonacci(0)) # should print 0
print(fibonacci(1)) # should print 1
print(fibonacci(2)) # should print 1
print(fibonacci(3)) # should print 2
print(fibonacci(4)) # should print 3
```
Each of these lines will call the `fibonacci` function with a different input value,

In [20]:
user_prompt = """ 
write A helper function that gets a user prompt and a system prompt and a model name and creates a model with ollama. 
use langchain ollama library .
the function should turn stream as True and return the final response text
"""


In [21]:
model_name = "codellama:13b"
response  = ollama_code(user_prompt, system_prompt, model_name)

# Import the necessary libraries
import ollamalib
from ollamalib import *

# Define the helper function
def create_model(user_prompt, system_prompt, model_name):
    # Initialize a new Ollama object
    ollama = Ollama()
    
    # Set the user prompt and system prompt for the model
    ollama.set_prompts(user_prompt, system_prompt)
    
    # Create a new model with the specified name
    model = ollama.create_model(model_name)
    
    # Set the stream to True
    model.stream = True
    
    # Return the final response text
    return model.final_response()

# Test the helper function
print("Testing the create_model function:")
user_prompt = "What is your name?"
system_prompt = "Hello there, my name is LLaMA!"
model_name = "test_model"
response = create_model(user_prompt, system_prompt, model_name)
print("Response:", response)